In [1]:
import pandas as pd
from pathlib import Path

In [2]:
Catholic_file = 'breathdata/augment_train.csv'
df = pd.read_csv(Catholic_file)
df

,filename,category,class
0,0001-1.wav,wheezing,1
1,0001-2.wav,healthy,0
2,0001-3.wav,wheezing,1
3,0001-4.wav,wheezing,1
4,0002-1.wav,healthy,0
...,...,...,...
1465,minus_0614-3.wav,wheezing,1
1466,minus_0614-4.wav,wheezing,1
1467,minus_0615-1.wav,wheezing,1
1468,minus_0615-2.wav,wheezing,1


In [3]:
df['relative_path'] = '/' + df['filename'].astype(str)
df = df[['relative_path', 'class']]
df.head()

,relative_path,class
0,/0001-1.wav,1
1,/0001-2.wav,0
2,/0001-3.wav,1
3,/0001-4.wav,1
4,/0002-1.wav,0


In [4]:
data_path = 'breathdata/train'

In [5]:
import math, random
import torch
import torchaudio
from torchaudio import transforms
from IPython.display import Audio

class Breath_sound_Util():
  
  def open(audio_file):
    sig, sr = torchaudio.load(audio_file)

    return (sig, sr)

  def resample(aud, newsr):
    sig, sr = aud
    
    if (sr == newsr):
     return aud

    num_channels = sig.shape[0]
    resig = torchaudio.transforms.Resample(sr, newsr)(sig[:1,:])
    if (num_channels > 1):
      retwo = torchaudio.transforms.Resample(sr, newsr)(sig[1:,:])
      resig = torch.cat([resig, retwo])

    return ((resig, newsr))
  

  def pad(aud, max_ms):
    sig, sr = aud
    num_rows, sig_len = sig.shape
    max_len = sr//1000 * max_ms
   
    if (sig_len > max_len):
      sig = sig[:,:max_len]
 
    elif (sig_len < max_len):

      repeated = []
      repeated.append(sig)
      required_len = max_len - sig_len

      while required_len > sig_len : 
        repeated.append(sig)
        require_len -= sig_len
      repeated.append(sig[:, :required_len])
 
      sig = torch.cat(repeated, 1)

    return (sig, sr)


  def spectro_gram(aud, n_mels=64, n_fft=1024, hop_len=None):
    sig,sr = aud
    top_db = 80
    
    spec = transforms.MelSpectrogram(sr, n_fft=n_fft, hop_length=hop_len, n_mels=n_mels)(sig)
    spec = transforms.AmplitudeToDB(top_db=top_db)(spec)
    return (spec)

  def spectro_augment(spec, max_mask_pct=0.1, n_freq_masks=1, n_time_masks=1):
    _, n_mels, n_steps = spec.shape
    mask_value = spec.mean()
    aug_spec = spec

    freq_mask_param = max_mask_pct * n_mels
    for _ in range(n_freq_masks):
      aug_spec = transforms.FrequencyMasking(freq_mask_param)(aug_spec, mask_value)

    time_mask_param = max_mask_pct * n_steps
    for _ in range(n_time_masks):
      aug_spec = transforms.TimeMasking(time_mask_param)(aug_spec, mask_value)

    return aug_spec
    print(aug_spec)

In [6]:
from torch.utils.data import DataLoader, Dataset, random_split
import torchaudio
import torchvision

class breathDS(Dataset):
    
  def __init__(self, df, data_path):
    self.df = df
    self.data_path = str(data_path)
    self.duration = 4000
    self.sr = 48000
            
  def __len__(self):
    return len(self.df)    
    
  def __getitem__(self, idx):
    audio_file = self.data_path + self.df.loc[idx, 'relative_path']
    class_id = self.df.loc[idx, 'class']
    aud = Breath_sound_Util.open(audio_file)
    reaud = Breath_sound_Util.resample(aud, self.sr)
    dur_aud = Breath_sound_Util.pad(reaud, self.duration)
    sgram = Breath_sound_Util.spectro_gram(dur_aud, n_mels=64, n_fft=1024, hop_len=None)
    aug_sgram = Breath_sound_Util.spectro_augment(sgram, max_mask_pct=0.1, n_freq_masks=2, n_time_masks=2)
    #aug_sgram = aug_sgram.view([-1, 64])
    
    
    return aug_sgram, class_id

In [7]:
from torch.utils.data import random_split

brds = breathDS(df, data_path)

In [8]:
train_dl = torch.utils.data.DataLoader(brds, batch_size=16, shuffle=True)

In [9]:
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from pytorch_utils import do_mixup, interpolate, pad_framewise_output

In [22]:
'''import torch
from torch import nn
import torch.nn.functional as F

PRETRAINED_CHECKPOINT_PATH="/home/nextgen/Desktop/braeth/Cnn14_mAP=0.431.pth"   # Trained by a later code version, achieves higher mAP than the paper.
MODEL_TYPE="Cnn14"

def init_layer(layer):
    """Initialize a Linear or Convolutional layer. """
    nn.init.xavier_uniform_(layer.weight)

    if hasattr(layer, 'bias'):
        if layer.bias is not None:
            layer.bias.data.fill_(0.)
            
def init_bn(bn):
    """Initialize a Batchnorm layer. """
    bn.bias.data.fill_(0.)
    bn.weight.data.fill_(1.)
    
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):

        super(ConvBlock, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=in_channels,
                               out_channels=out_channels,
                               kernel_size=(3, 3), stride=(1, 1),
                               padding=(1, 1), bias=False)

        self.conv2 = nn.Conv2d(in_channels=out_channels,
                               out_channels=out_channels,
                               kernel_size=(3, 3), stride=(1, 1),
                               padding=(1, 1), bias=False)

        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.init_weight()

    def init_weight(self):
        init_layer(self.conv1)
        init_layer(self.conv2)
        init_bn(self.bn1)
        init_bn(self.bn2)

    def forward(self, input, pool_size=(2, 2), pool_type='avg'):
        x = input
        x = F.relu_(self.bn1(self.conv1(x)))
        x = F.relu_(self.bn2(self.conv2(x)))
        if pool_type == 'max':
            x = F.max_pool2d(x, kernel_size=pool_size)
        elif pool_type == 'avg':
            x = F.avg_pool2d(x, kernel_size=pool_size)
        elif pool_type == 'avg+max':
            x1 = F.avg_pool2d(x, kernel_size=pool_size)
            x2 = F.max_pool2d(x, kernel_size=pool_size)
            x = x1 + x2
        else:
            raise Exception('Incorrect argument!')

        return x
        
class Cnn14(nn.Module):
    def __init__(self, classes_num=527):

        super(Cnn14, self).__init__()

        self.bn0 = nn.BatchNorm2d(64)

        self.conv_block1 = ConvBlock(in_channels=1, out_channels=64)
        self.conv_block2 = ConvBlock(in_channels=64, out_channels=128)
        self.conv_block3 = ConvBlock(in_channels=128, out_channels=256)
        self.conv_block4 = ConvBlock(in_channels=256, out_channels=512)
        self.conv_block5 = ConvBlock(in_channels=512, out_channels=1024)
        self.conv_block6 = ConvBlock(in_channels=1024, out_channels=2048)

        self.fc1 = nn.Linear(2048, 2048, bias=True)
        self.fc_audioset = nn.Linear(2048, classes_num, bias=True)

        self.init_weight()

    def init_weight(self):
        init_bn(self.bn0)

    def forward(self, input):

        x = input #unsqueeze(1)   # (batch_size, 1, time_steps, mel_bins)
        print(x.shape)
        #x = torch.unsqueeze(x, 1)
        print(x.shape)
        x = x.transpose(2, 3)
        print(x.shape)
        x = x.transpose(1, 3)
        x = self.bn0(x)
        x = x.transpose(1, 3)
        print(x.shape)

        x = self.conv_block1(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block2(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block3(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block4(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block5(x, pool_size=(1, 1), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block6(x, pool_size=(1, 1), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)     #(batch_size, 2048, T/16, mel_bins/16)
        

        return x

class Transfer_Cnn14(nn.Module):
    def __init__(self, pretrain_checkpoint=None):
        super(Transfer_Cnn14, self).__init__()
        classes_num = 2
        audioset_classes_num = 527
        self.base = Cnn14(audioset_classes_num)

        #self.init_weights()

        if pretrain_checkpoint:
            self.load_from_pretrain(pretrain_checkpoint)
            
        self.base.fc_audioset = nn.Linear(2048, classes_num, bias=True)
        # 안쓰이는이유는 multi-class clasification을 생략하기 때문,


    #def init_weights(self):
        #init_layer(self.fc_transfer)

    def load_from_pretrain(self, pretrained_checkpoint):
        pretrained_checkpoint="/home/nextgen/Desktop/braeth/Cnn14_mAP=0.431.pth"
        checkpoint = torch.load(pretrained_checkpoint)
        self.base.load_state_dict(checkpoint['model'])

    def forward(self, input):
        """Input: (batch_size, data_length)
        """
        output = self.base(input)

        #embedding = output_dict['embedding']
        #clipwise_output = output_dict['clipwise_output']

        return output #, clipwise_output
    
Model1 = Transfer_Cnn14()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
Model1 = Model1.to(device)
next(Model1.parameters()).device'''

device(type='cuda', index=0)

In [11]:
import torch
from torch import nn
import torch.nn.functional as F

PRETRAINED_CHECKPOINT_PATH="/home/nextgen/Desktop/braeth/Cnn14_mAP=0.431.pth"   # Trained by a later code version, achieves higher mAP than the paper.
MODEL_TYPE="Cnn14"

def init_layer(layer):
    """Initialize a Linear or Convolutional layer. """
    nn.init.xavier_uniform_(layer.weight)

    if hasattr(layer, 'bias'):
        if layer.bias is not None:
            layer.bias.data.fill_(0.)
            
def init_bn(bn):
    """Initialize a Batchnorm layer. """
    bn.bias.data.fill_(0.)
    bn.weight.data.fill_(1.)
    
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):

        super(ConvBlock, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=in_channels,
                               out_channels=out_channels,
                               kernel_size=(3, 3), stride=(1, 1),
                               padding=(1, 1), bias=False)

        self.conv2 = nn.Conv2d(in_channels=out_channels,
                               out_channels=out_channels,
                               kernel_size=(3, 3), stride=(1, 1),
                               padding=(1, 1), bias=False)

        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.init_weight()

    def init_weight(self):
        init_layer(self.conv1)
        init_layer(self.conv2)
        init_bn(self.bn1)
        init_bn(self.bn2)

    def forward(self, input, pool_size=(2, 2), pool_type='avg'):
        x = input
        x = F.relu_(self.bn1(self.conv1(x)))
        x = F.relu_(self.bn2(self.conv2(x)))
        if pool_type == 'max':
            x = F.max_pool2d(x, kernel_size=pool_size)
        elif pool_type == 'avg':
            x = F.avg_pool2d(x, kernel_size=pool_size)
        elif pool_type == 'avg+max':
            x1 = F.avg_pool2d(x, kernel_size=pool_size)
            x2 = F.max_pool2d(x, kernel_size=pool_size)
            x = x1 + x2
        else:
            raise Exception('Incorrect argument!')

        return x

In [16]:
class Cnn14(nn.Module):
    def __init__(self, classes_num):
        
        super(Cnn14, self).__init__()

        self.bn0 = nn.BatchNorm2d(64)

        self.conv_block1 = ConvBlock(in_channels=1, out_channels=64)
        self.conv_block2 = ConvBlock(in_channels=64, out_channels=128)
        self.conv_block3 = ConvBlock(in_channels=128, out_channels=256)
        self.conv_block4 = ConvBlock(in_channels=256, out_channels=512)
        self.conv_block5 = ConvBlock(in_channels=512, out_channels=1024)
        self.conv_block6 = ConvBlock(in_channels=1024, out_channels=2048)

        self.fc1 = nn.Linear(2048, 2048, bias=True)
        self.fc_audioset = nn.Linear(2048, classes_num, bias=True)
        
        self.init_weight()

    def init_weight(self):
        init_bn(self.bn0)
        init_layer(self.fc1)
        init_layer(self.fc_audioset)
 
    def forward(self, input, mixup_lambda=None):
        """
        Input: (batch_size, data_length)"""

        #x = self.spectrogram_extractor(input)   # (batch_size, 1, time_steps, freq_bins)
        #x = self.logmel_extractor(x)    # (batch_size, 1, time_steps, mel_bins)
        
        #x = input.unsqueeze(0)
        x = input.transpose(1, 3)
        print(x.shape)
        x = x.transpose(1, 2)
        x = self.bn0(x)
        print(x.shape)
        x = x.transpose(1, 3)
        print(x.shape)


        # Mixup on spectrogram
        if self.training and mixup_lambda is not None:
            x = do_mixup(x, mixup_lambda)

        x = self.conv_block1(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block2(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block3(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block4(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block5(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block6(x, pool_size=(1, 1), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = torch.mean(x, dim=3)
        
        (x1, _) = torch.max(x, dim=2)
        x2 = torch.mean(x, dim=2)
        x = x1 + x2
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu_(self.fc1(x))
        embedding = F.dropout(x, p=0.5, training=self.training)
        clipwise_output = torch.sigmoid(self.fc_audioset(x))
        
        output_dict = {'clipwise_output': clipwise_output, 'embedding': embedding}

        return output_dict


class Transfer_Cnn14(nn.Module):
    def __init__(self,classes_num, pretrain_checkpoint=None):
        super(Transfer_Cnn14, self).__init__()
        classes_num = 2
        audioset_classes_num = 527
        self.base = Cnn14(audioset_classes_num)

        #self.init_weights()

        if pretrain_checkpoint:
            self.load_from_pretrain(pretrain_checkpoint)
            
        self.base.fc_audioset = nn.Linear(2048, classes_num, bias=True)
        # 안쓰이는이유는 multi-class clasification을 생략하기 때문,


    #def init_weights(self):
        #init_layer(self.fc_transfer)

    def load_from_pretrain(self, pretrained_checkpoint):
        pretrained_checkpoint="Cnn14_mAP=0.431.pth"
        checkpoint = torch.load(pretrained_checkpoint)
        checkpoint['model'].pop('spectrogram_extractor.stft.conv_imag.weight')
        checkpoint['model'].pop('spectrogram_extractor.stft.conv_real.weight')
        checkpoint['model'].pop('logmel_extractor.melW')
        self.base.load_state_dict(checkpoint['model'])
    
    def forward(self, input, mixup_lambda=None):
        """Input: (batch_size, data_length)
        """
        output_dict = self.base(input, mixup_lambda)
        embedding = output_dict['embedding']

        clipwise_output =  torch.log_softmax(self.base.fc_audioset(embedding), dim=-1)
        #output_dict['clipwise_output'] = clipwise_output
 
        return clipwise_output

In [17]:
Transfer_Cnn14(classes_num=2,pretrain_checkpoint="Cnn14_mAP=0.431.pth")

Transfer_Cnn14(
  (base): Cnn14(
    (bn0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv_block1): ConvBlock(
      (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_block2): ConvBlock(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_block3): ConvBlock(
      (conv1): Conv2d(128,

In [18]:
Model1=Transfer_Cnn14(classes_num=2,pretrain_checkpoint="Cnn14_mAP=0.431.pth")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
Model1 = Model1.to(device)
next(Model1.parameters()).device

device(type='cuda', index=0)

In [17]:
import numpy as np
for epoch in range(2):
    for i, data in enumerate(train_dl, 0):
        inputs, labels = data
        inputs = np.array(inputs)
        print(inputs.shape)
        print(f'Epoch: {i} | Inputs {inputs} | Labels {labels}')

(16, 1, 64, 376)
Epoch: 0 | Inputs [[[[-40.754097  -11.720562  -18.38048   ...  -6.837266   -4.8647327
     -5.074279 ]
   [-25.343134  -16.523022  -23.623121  ...  -9.705797   -9.1601
     -9.577418 ]
   [-22.40943   -23.710709  -30.940493  ... -15.608305  -15.211991
    -15.489355 ]
   ...
   [-67.5783    -67.5783    -67.5783    ... -67.5783    -67.5783
    -67.5783   ]
   [-67.5783    -67.5783    -67.5783    ... -67.5783    -67.5783
    -67.5783   ]
   [-67.5783    -67.5783    -67.5783    ... -67.5783    -67.5783
    -67.5783   ]]]


 [[[-12.040726   -4.9154253  -1.4111542 ... -14.54794    -3.8779423
     -4.230362 ]
   [-18.054626   -8.84503    -7.1323757 ... -19.79359    -9.698307
    -10.837201 ]
   [-30.159636  -16.00459   -17.282757  ... -21.935055  -15.883933
    -17.058924 ]
   ...
   [-61.761513  -61.761513  -61.761513  ... -61.761513  -61.761513
    -61.761513 ]
   [-61.761513  -61.761513  -61.761513  ... -61.761513  -61.761513
    -61.761513 ]
   [-77.98657   -75.64758   -

(16, 1, 64, 376)
Epoch: 3 | Inputs [[[[-4.57946587e+01 -3.81000671e+01 -3.81000671e+01 ... -3.64739132e+00
     2.98208761e+00  7.44584918e-01]
   [-4.57946587e+01 -3.81000671e+01 -3.81000671e+01 ... -6.95054913e+00
     9.32911225e-03 -4.93973207e+00]
   [-4.57946587e+01 -3.81000671e+01 -3.81000671e+01 ... -9.10247421e+00
    -4.23427296e+00 -1.20279350e+01]
   ...
   [-4.57946587e+01 -3.81000671e+01 -3.81000671e+01 ... -4.57946587e+01
    -4.57946587e+01 -4.57946587e+01]
   [-4.57946587e+01 -3.81000671e+01 -3.81000671e+01 ... -4.57946587e+01
    -4.57946587e+01 -4.57946587e+01]
   [-4.57946587e+01 -3.81000671e+01 -3.81000671e+01 ... -4.57946587e+01
    -4.57946587e+01 -4.57946587e+01]]]


 [[[-9.55676842e+00 -1.01056337e+01 -3.71691656e+00 ... -1.88539467e+01
    -2.37118835e+01 -4.43067360e+01]
   [-9.68588924e+00 -1.38018942e+01 -9.23295307e+00 ... -2.19663525e+01
    -2.91933804e+01 -2.73299427e+01]
   [-7.69474936e+00 -2.01803207e+01 -1.75408268e+01 ... -1.98979874e+01
    -1.848

(16, 1, 64, 376)
Epoch: 6 | Inputs [[[[-25.161219  -16.474657   -9.639441  ... -24.847755   -8.630866
    -19.040752 ]
   [-28.167665  -21.518736  -15.8409395 ... -18.060516   -9.002169
    -17.331028 ]
   [-32.390408  -27.923088  -29.790192  ... -16.801527  -13.407377
    -21.588156 ]
   ...
   [-75.94563   -78.45209   -78.45209   ... -78.45209   -78.45209
    -78.45209  ]
   [-78.45209   -78.45209   -78.45209   ... -78.45209   -78.45209
    -78.45209  ]
   [-78.45209   -78.45209   -78.45209   ... -78.45209   -78.45209
    -78.45209  ]]]


 [[[-40.699066  -17.904818  -12.804331  ... -22.016794  -35.285618
    -22.893852 ]
   [-25.34321   -22.97514   -19.403803  ... -25.638603  -31.335789
    -29.477903 ]
   [-28.706528  -25.99685   -26.336525  ... -31.496479  -34.211315
    -39.49461  ]
   ...
   [-77.32134   -77.32134   -77.32134   ... -77.32134   -77.32134
    -77.32134  ]
   [-77.32134   -77.32134   -77.32134   ... -77.32134   -77.32134
    -77.32134  ]
   [-77.32134   -77.32134   

(16, 1, 64, 376)
Epoch: 9 | Inputs [[[[ -2.5348413  -10.839921   -10.522876   ...   1.2485204
    -10.256349    -7.425893  ]
   [-41.549854   -41.549854   -41.549854   ... -41.549854
    -41.549854   -41.549854  ]
   [-41.549854   -41.549854   -41.549854   ... -41.549854
    -41.549854   -41.549854  ]
   ...
   [-42.993164   -67.86102    -67.86102    ... -67.86102
    -67.86102    -67.86102   ]
   [-42.88608    -62.139595   -63.04695    ... -67.86102
    -67.86102    -67.86102   ]
   [-42.796883   -64.82377    -65.73834    ... -67.86102
    -67.86102    -67.86102   ]]]


 [[[-30.007793   -29.141201   -27.55662    ... -18.397696
      1.9770668   -4.385944  ]
   [-25.249664   -22.289858   -19.539257   ...  -7.5567675
      3.0527835    3.0603375 ]
   [-22.695417   -14.713974   -19.079874   ...  -0.59906816
      0.6965642   -1.059666  ]
   ...
   [-49.67857    -49.774338   -49.568195   ... -48.823208
    -49.490173   -46.643692  ]
   [-45.56234    -48.137074   -47.68313    ... -46.64050

(16, 1, 64, 376)
Epoch: 12 | Inputs [[[[  2.4698148    1.7191416    2.9808717  ...   2.839027
     -8.538731    -1.8642043 ]
   [ -4.0046935    3.815179     6.374412   ...   4.5635805
     -4.110404     0.40848655]
   [ 10.797721     5.238456     4.4588294  ...   4.2827806
      1.140401     3.002076  ]
   ...
   [-48.08064    -62.310143   -62.310143   ... -62.310143
    -62.310143   -59.637394  ]
   [-47.705276   -57.18676    -57.8263     ... -57.959934
    -62.310143   -57.2221    ]
   [-47.81674    -58.60037    -58.93493    ... -59.197838
    -62.310143   -57.763386  ]]]


 [[[-13.243975    -7.0227137   -7.932613   ... -13.309576
    -25.565338   -23.350813  ]
   [-13.511293   -12.644274   -13.590979   ... -19.19998
    -27.722408   -26.864508  ]
   [-17.879824   -22.570503   -23.550987   ... -30.826834
    -32.796608   -31.936275  ]
   ...
   [-85.94125    -85.94125    -85.94125    ... -85.94125
    -85.94125    -85.94125   ]
   [-80.995834   -76.31463    -78.77589    ... -82.39539

...

(16, 1, 64, 376)
Epoch: 30 | Inputs [[[[-23.827656  -12.852584  -21.948895  ... -20.898808  -18.676659
    -13.945236 ]
   [-26.81074   -18.007204  -25.92772   ... -26.080223  -25.043972
    -20.137976 ]
   [-32.73073   -26.967222  -31.89371   ... -33.99554   -40.46681
    -34.18703  ]
   ...
   [-72.57472   -72.57472   -72.57472   ... -72.57472   -72.57472
    -72.57472  ]
   [-72.57472   -72.57472   -72.57472   ... -72.57472   -72.57472
    -72.57472  ]
   [-72.57472   -72.57472   -72.57472   ... -72.57472   -72.57472
    -72.57472  ]]]


 [[[-14.337039   -1.4975243  -2.674047  ...   1.2560842   3.9704525
     -4.1061244]
   [ -6.3989706   1.1458797  -6.161603  ...   9.10933    -2.2421954
      6.6079826]
   [ -8.333226   -1.6607518  -9.961177  ...  15.942079   14.979485
     15.68253  ]
   ...
   [-35.159504  -35.159504  -35.159504  ... -35.159504  -35.159504
    -35.159504 ]
   [-35.159504  -35.159504  -35.159504  ... -35.159504  -35.159504
    -35.159504 ]
   [-45.068665  -45.0686

(16, 1, 64, 376)
Epoch: 33 | Inputs [[[[-4.7235332e+00  3.0324175e+00 -1.2558192e+01 ...  4.1933417e+00
     8.0241770e-01 -1.5807114e+01]
   [-7.3935261e+00 -1.8438092e-01 -1.5368923e+01 ... -1.8477863e+00
    -4.4220920e+00 -6.7941098e+00]
   [-6.4531355e+00 -6.6027660e+00 -1.6381184e+01 ... -1.4708127e+01
    -1.3380032e+01 -9.1525574e+00]
   ...
   [-6.6267593e+01 -6.6267593e+01 -6.6267593e+01 ... -6.6267593e+01
    -6.6267593e+01 -6.6267593e+01]
   [-6.6267593e+01 -6.6267593e+01 -6.6267593e+01 ... -6.5138573e+01
    -6.6267593e+01 -6.6267593e+01]
   [-6.6267593e+01 -6.6267593e+01 -6.6267593e+01 ... -6.5441460e+01
    -6.6267593e+01 -6.6267593e+01]]]


 [[[-5.0322418e+01 -3.3480450e+01 -3.2550198e+01 ... -2.1566618e+01
    -3.2808556e+01 -2.0468769e+01]
   [-5.5331329e+01 -3.4144081e+01 -2.3794571e+01 ... -2.4754059e+01
    -3.3324932e+01 -2.4501663e+01]
   [-5.7819206e+01 -3.4187614e+01 -2.2229023e+01 ... -3.0439468e+01
    -3.2937279e+01 -3.1886143e+01]
   ...
   [-7.3931480e+01 

(16, 1, 64, 376)
Epoch: 36 | Inputs [[[[-16.800472  -17.866217  -16.859266  ... -15.565301  -13.144173
    -14.058127 ]
   [-22.890646  -23.804947  -19.455765  ... -21.86956   -15.674686
    -20.286049 ]
   [-35.87339   -27.234632  -21.934093  ... -33.10965   -21.095997
    -25.579033 ]
   ...
   [-64.7699    -80.18616   -80.18616   ... -80.18616   -80.18616
    -66.69475  ]
   [-65.08857   -80.18616   -80.18616   ... -80.18616   -80.18616
    -67.56942  ]
   [-65.19478   -80.18616   -80.18616   ... -80.18616   -80.18616
    -67.35894  ]]]


 [[[-76.14468   -64.85238   -49.237118  ... -11.768962  -15.311148
    -13.407572 ]
   [-76.14468   -65.44484   -52.714035  ... -17.79055   -17.413448
    -16.70856  ]
   [-76.14468   -66.71178   -56.80847   ... -29.55076   -22.204695
    -21.98511  ]
   ...
   [-76.14468   -76.14468   -76.14468   ... -76.14468   -76.14468
    -76.14468  ]
   [-76.14468   -76.14468   -76.14468   ... -76.14468   -76.14468
    -76.14468  ]
   [-76.14468   -76.14468  

(16, 1, 64, 376)
Epoch: 39 | Inputs [[[[-64.9833     -54.107353   -48.52987    ...  -4.3750005
     -3.3697777  -27.239561  ]
   [-64.9833     -56.27923    -55.151714   ...  -9.574635
     -6.124106   -11.988035  ]
   [-64.9833     -59.9992     -64.9833     ... -17.484173
    -12.171889   -15.9420395 ]
   ...
   [-64.9833     -64.9833     -64.9833     ... -64.9833
    -64.9833     -64.9833    ]
   [-64.9833     -64.9833     -64.9833     ... -64.9833
    -64.9833     -64.9833    ]
   [-64.9833     -64.9833     -64.9833     ... -64.9833
    -64.9833     -64.9833    ]]]


 [[[-11.32771    -11.947943   -18.187046   ... -16.955297
    -10.752251    -8.860658  ]
   [ -9.72886    -13.633681   -24.309034   ... -22.30201
    -15.240261   -14.833717  ]
   [-13.806761   -18.76776    -34.45018    ... -30.822298
    -23.271017   -25.788939  ]
   ...
   [-64.59819    -64.59819    -64.59819    ... -64.59819
    -64.59819    -64.59819   ]
   [-64.59819    -64.59819    -64.59819    ... -64.59819
    -6

KeyboardInterrupt: 

In [19]:
from sklearn.metrics import confusion_matrix
import numpy as np
import sklearn.metrics as metrics
from tqdm import tqdm

def training(model, train_dl, num_epochs):
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
  scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.001,
                                                steps_per_epoch=int(len(train_dl)),
                                                epochs=num_epochs,
                                                anneal_strategy='linear')

  for epoch in tqdm(range(num_epochs)):
    running_loss = 0.0
    correct_prediction = 0
    total_prediction = 0

    for i, data in enumerate(train_dl):
        inputs, labels = data[0].to(device), data[1].to(device)
        inputs_m, inputs_s = inputs.mean(), inputs.std()
        inputs = (inputs - inputs_m) / inputs_s
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()
        running_loss += loss.item()
        _, prediction = torch.max(outputs,1)
        correct_prediction += (prediction == labels).sum().item()
        total_prediction += prediction.shape[0]
        

    num_batches = len(train_dl)
    avg_loss = running_loss / num_batches
    acc = correct_prediction/total_prediction
    print(f'Epoch: {epoch}, Loss: {avg_loss:.4f}, Accuracy: {acc:.4f}')

  print('Finished Training')
    
num_epochs=52
training(Model1, train_dl, num_epochs)

  0%|          | 0/52 [00:00<?, ?it/s]

torch.Size([16, 376, 64, 1])
torch.Size([16, 64, 376, 1])
torch.Size([16, 1, 376, 64])
torch.Size([16, 376, 64, 1])
torch.Size([16, 64, 376, 1])
torch.Size([16, 1, 376, 64])
torch.Size([16, 376, 64, 1])
torch.Size([16, 64, 376, 1])
torch.Size([16, 1, 376, 64])
torch.Size([16, 376, 64, 1])
torch.Size([16, 64, 376, 1])
torch.Size([16, 1, 376, 64])
torch.Size([16, 376, 64, 1])
torch.Size([16, 64, 376, 1])
torch.Size([16, 1, 376, 64])
torch.Size([16, 376, 64, 1])
torch.Size([16, 64, 376, 1])
torch.Size([16, 1, 376, 64])
torch.Size([16, 376, 64, 1])
torch.Size([16, 64, 376, 1])
torch.Size([16, 1, 376, 64])
torch.Size([16, 376, 64, 1])
torch.Size([16, 64, 376, 1])
torch.Size([16, 1, 376, 64])
torch.Size([16, 376, 64, 1])
torch.Size([16, 64, 376, 1])
torch.Size([16, 1, 376, 64])
torch.Size([16, 376, 64, 1])
torch.Size([16, 64, 376, 1])
torch.Size([16, 1, 376, 64])
torch.Size([16, 376, 64, 1])
torch.Size([16, 64, 376, 1])
torch.Size([16, 1, 376, 64])
torch.Size([16, 376, 64, 1])
torch.Size([16

  2%|▏         | 1/52 [00:38<32:58, 38.80s/it]

torch.Size([14, 376, 64, 1])
torch.Size([14, 64, 376, 1])
torch.Size([14, 1, 376, 64])
Epoch: 0, Loss: 0.6560, Accuracy: 0.6435
torch.Size([16, 376, 64, 1])
torch.Size([16, 64, 376, 1])
torch.Size([16, 1, 376, 64])
torch.Size([16, 376, 64, 1])
torch.Size([16, 64, 376, 1])
torch.Size([16, 1, 376, 64])
torch.Size([16, 376, 64, 1])
torch.Size([16, 64, 376, 1])
torch.Size([16, 1, 376, 64])
torch.Size([16, 376, 64, 1])
torch.Size([16, 64, 376, 1])
torch.Size([16, 1, 376, 64])
torch.Size([16, 376, 64, 1])
torch.Size([16, 64, 376, 1])
torch.Size([16, 1, 376, 64])
torch.Size([16, 376, 64, 1])
torch.Size([16, 64, 376, 1])
torch.Size([16, 1, 376, 64])
torch.Size([16, 376, 64, 1])
torch.Size([16, 64, 376, 1])
torch.Size([16, 1, 376, 64])
torch.Size([16, 376, 64, 1])
torch.Size([16, 64, 376, 1])
torch.Size([16, 1, 376, 64])
torch.Size([16, 376, 64, 1])
torch.Size([16, 64, 376, 1])
torch.Size([16, 1, 376, 64])
torch.Size([16, 376, 64, 1])
torch.Size([16, 64, 376, 1])
torch.Size([16, 1, 376, 64])
to

  4%|▍         | 2/52 [01:16<31:57, 38.36s/it]

...

torch.Size([14, 376, 64, 1])
torch.Size([14, 64, 376, 1])
torch.Size([14, 1, 376, 64])
Epoch: 49, Loss: 0.0030, Accuracy: 1.0000
torch.Size([16, 376, 64, 1])
torch.Size([16, 64, 376, 1])
torch.Size([16, 1, 376, 64])
torch.Size([16, 376, 64, 1])
torch.Size([16, 64, 376, 1])
torch.Size([16, 1, 376, 64])
torch.Size([16, 376, 64, 1])
torch.Size([16, 64, 376, 1])
torch.Size([16, 1, 376, 64])
torch.Size([16, 376, 64, 1])
torch.Size([16, 64, 376, 1])
torch.Size([16, 1, 376, 64])
torch.Size([16, 376, 64, 1])
torch.Size([16, 64, 376, 1])
torch.Size([16, 1, 376, 64])
torch.Size([16, 376, 64, 1])
torch.Size([16, 64, 376, 1])
torch.Size([16, 1, 376, 64])
torch.Size([16, 376, 64, 1])
torch.Size([16, 64, 376, 1])
torch.Size([16, 1, 376, 64])
torch.Size([16, 376, 64, 1])
torch.Size([16, 64, 376, 1])
torch.Size([16, 1, 376, 64])
torch.Size([16, 376, 64, 1])
torch.Size([16, 64, 376, 1])
torch.Size([16, 1, 376, 64])
torch.Size([16, 376, 64, 1])
torch.Size([16, 64, 376, 1])
torch.Size([16, 1, 376, 64])
t

 98%|█████████▊| 51/52 [32:35<00:39, 39.23s/it]

torch.Size([14, 376, 64, 1])
torch.Size([14, 64, 376, 1])
torch.Size([14, 1, 376, 64])
Epoch: 50, Loss: 0.0019, Accuracy: 0.9993
torch.Size([16, 376, 64, 1])
torch.Size([16, 64, 376, 1])
torch.Size([16, 1, 376, 64])
torch.Size([16, 376, 64, 1])
torch.Size([16, 64, 376, 1])
torch.Size([16, 1, 376, 64])
torch.Size([16, 376, 64, 1])
torch.Size([16, 64, 376, 1])
torch.Size([16, 1, 376, 64])
torch.Size([16, 376, 64, 1])
torch.Size([16, 64, 376, 1])
torch.Size([16, 1, 376, 64])
torch.Size([16, 376, 64, 1])
torch.Size([16, 64, 376, 1])
torch.Size([16, 1, 376, 64])
torch.Size([16, 376, 64, 1])
torch.Size([16, 64, 376, 1])
torch.Size([16, 1, 376, 64])
torch.Size([16, 376, 64, 1])
torch.Size([16, 64, 376, 1])
torch.Size([16, 1, 376, 64])
torch.Size([16, 376, 64, 1])
torch.Size([16, 64, 376, 1])
torch.Size([16, 1, 376, 64])
torch.Size([16, 376, 64, 1])
torch.Size([16, 64, 376, 1])
torch.Size([16, 1, 376, 64])
torch.Size([16, 376, 64, 1])
torch.Size([16, 64, 376, 1])
torch.Size([16, 1, 376, 64])
t

100%|██████████| 52/52 [33:14<00:00, 38.35s/it]

torch.Size([14, 376, 64, 1])
torch.Size([14, 64, 376, 1])
torch.Size([14, 1, 376, 64])
Epoch: 51, Loss: 0.0048, Accuracy: 0.9986
Finished Training


In [20]:
Catholic_file1 = 'breathdata/test.csv'
df1 = pd.read_csv(Catholic_file1)
df1

,filename,category,class
0,0003-1.wav,healthy,0
1,0004-1.wav,healthy,0
2,0007-1.wav,healthy,0
3,0009-1.wav,healthy,0
4,0009-4.wav,healthy,0
5,0012-2.wav,healthy,0
6,0013-2.wav,healthy,0
7,0015-1.wav,healthy,0
8,0020-1.wav,wheezing,1
9,0020-3.wav,wheezing,1


In [21]:
df1['relative_path'] = '/' + df1['filename'].astype(str)
df1 = df1[['relative_path', 'class']]
df1.head()

,relative_path,class
0,/0003-1.wav,0
1,/0004-1.wav,0
2,/0007-1.wav,0
3,/0009-1.wav,0
4,/0009-4.wav,0


In [22]:
data_path = 'breathdata/test'

In [23]:
class Breath_sound_Util1():
  
  def open(audio_file):
    sig, sr = torchaudio.load(audio_file)
    
    return (sig, sr)

  def resample(aud, newsr):
    sig, sr = aud
    
    if (sr == newsr):
     return aud

    num_channels = sig.shape[0]
    resig = torchaudio.transforms.Resample(sr, newsr)(sig[:1,:])
    if (num_channels > 1):
      retwo = torchaudio.transforms.Resample(sr, newsr)(sig[1:,:])
      resig = torch.cat([resig, retwo])

    return ((resig, newsr))
  

  def pad(aud, max_ms):
    sig, sr = aud
    num_rows, sig_len = sig.shape
    max_len = sr//1000 * max_ms
   
    if (sig_len > max_len):
      sig = sig[:,:max_len]
 
    elif (sig_len < max_len):

      repeated = []
      repeated.append(sig)
      required_len = max_len - sig_len

      while required_len > sig_len : 
        repeated.append(sig)
        require_len -= sig_len
      repeated.append(sig[:, :required_len])
 
      sig = torch.cat(repeated, 1)

    return (sig, sr)

  def spectro_gram(aud, n_mels=64, n_fft=1024, hop_len=None):
    sig,sr = aud
    top_db = 80
    
    spec = transforms.MelSpectrogram(sr, n_fft=n_fft, hop_length=hop_len, n_mels=n_mels)(sig)
    spec = transforms.AmplitudeToDB(top_db=top_db)(spec)
    return (spec)

In [24]:
class breathDS1(Dataset):
    
  def __init__(self, df1, data_path):
    self.df1 = df1
    self.data_path = str(data_path)
    self.duration = 4000
    self.sr = 48000
            
  def __len__(self):
    return len(self.df1)    
    
  def __getitem__(self, idx):
    audio_file = self.data_path + self.df1.loc[idx, 'relative_path']
    class_id = self.df1.loc[idx, 'class']
    aud = Breath_sound_Util1.open(audio_file)
    reaud = Breath_sound_Util1.resample(aud, self.sr)
    dur_aud = Breath_sound_Util1.pad(reaud, self.duration)
    sgram = Breath_sound_Util1.spectro_gram(dur_aud, n_mels=64, n_fft=1024, hop_len=None)

    
    return sgram, class_id

In [25]:
brds1 = breathDS1(df1, data_path)

In [26]:
val_dl = torch.utils.data.DataLoader(brds1, batch_size=16, shuffle=True)

In [39]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import roc_curve, roc_auc_score

def inference (Model1, val_dl):
  correct_prediction = 0
  total_prediction = 0
  TP = 0 
  TN = 0
  FN = 0
  FP = 0
  a = []
  b = []
  with torch.no_grad():
    for i, data in enumerate(val_dl):
      
      inputs, labels = data[0].to(device), data[1].to(device)
      cpu_labels = data[1]
      inputs_m, inputs_s = inputs.mean(), inputs.std()
      inputs = (inputs - inputs_m) / inputs_s
      outputs = Model1(inputs)
 
      _, prediction = torch.max(outputs,1)
      
      cpu_prediction = prediction.to('cpu')
      correct_prediction += (prediction == labels).sum().item()
      total_prediction += prediction.shape[0]
          

      TP += ((prediction == 1) & (labels == 1)).sum()
# TN predict and label are both 0
      TN += ((prediction == 0) & (labels == 0)).sum()
# FN    predict 0 label 1
      FN += ((prediction == 0) & (labels == 1)).sum()
# FP    predict 1 label 0
      FP += ((prediction == 1) & (labels == 0)).sum()

      c = cpu_labels.tolist()
      a.extend(c)

      d = cpu_prediction.tolist()
      b.extend(d)
    
    # TP : 휘징인데 휘징이라고 판단(잘함)
    # FP : 헬시인데 휘징이라고 판단(못함)
    # TN : 헬시인데 헬시라고 판단(잘함)
    # FN : 휘징인데 헬시라고 판단(못함)
    
    # POSITIVE : 1(휘징)이라고 판단
    # NEGATIVE : 0(헬시)라고 판단
    # TRUE : 정답값 1(휘징)
    # FALSE : 정답값 0(헬시)
    
    # PRECISION : 휘징이라고 판단한 것 중 진짜 휘징
    # RECALL : 휘징인데 휘징이라고 잘 판단
    

  precision = TP / (TP + FP)
  recall = TP / (TP + FN)
  F1 = 2 * recall * precision / (recall + precision)

  acc = correct_prediction/total_prediction
  print(f'Accuracy: {acc:.4f}, Total items: {total_prediction}')
  print(f'precision: {precision:.4f}, F1: {F1:.4f}')
  print(f'recall: {recall:.4f}')
  target_names = ['healthy', 'wheezing']
  print(classification_report(a, b, target_names = target_names))
  print("AUC:{}".format(roc_auc_score(a, b)))

inference(Model1, val_dl)

torch.Size([16, 376, 64, 1])
torch.Size([16, 64, 376, 1])
torch.Size([16, 1, 376, 64])
torch.Size([16, 376, 64, 1])
torch.Size([16, 64, 376, 1])
torch.Size([16, 1, 376, 64])
torch.Size([16, 376, 64, 1])
torch.Size([16, 64, 376, 1])
torch.Size([16, 1, 376, 64])
torch.Size([5, 376, 64, 1])
torch.Size([5, 64, 376, 1])
torch.Size([5, 1, 376, 64])
Accuracy: 0.8491, Total items: 53
precision: 0.8000, F1: 0.7500
recall: 0.7059
              precision    recall  f1-score   support

     healthy       0.87      0.92      0.89        36
    wheezing       0.80      0.71      0.75        17

    accuracy                           0.85        53
   macro avg       0.83      0.81      0.82        53
weighted avg       0.85      0.85      0.85        53

AUC:0.8112745098039216
